In [2]:
# Part 1 

import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
with pm.Model() as logistic_model:

    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))

    linear_comb = pm.math.dot(X, betas)

    p = pm.math.sigmoid(linear_comb) 

    y_obs = pm.Bernoulli('y_obs', p=p, observed=y)
with logistic_model:
    idata = pm.sample()


In [13]:
# Part 2

import pymc as pm
import numpy as np
import arviz as az


np.random.seed(42)
n_samples = 100
X = np.random.normal(size=(n_samples, 1))
true_beta = 2.5
mu = X * true_beta
w = 1
nu = 4


y = np.random.normal(mu.flatten(), scale=w)

with pm.Model() as robust_model_with_lambda:

    beta = pm.Normal('beta', mu=0, sigma=10)
    nu = pm.Exponential('nu', 1/29) + 1
    lambda_i = pm.Gamma('lambda', alpha=nu/2, beta=nu/2, shape=n_samples)
    sigma_i = pm.Deterministic('sigma_i', 1 / (lambda_i))

    likelihood = pm.Normal('y', mu=X[:,0]*beta, sigma=sigma_i, observed=y)

    trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

lambda_posterior = trace.posterior['lambda'].values
lambda_mean = np.mean(lambda_posterior, axis=(0, 1))
outlier_threshold = np.quantile(lambda_mean, 0.05)  
outliers = np.where(lambda_mean < outlier_threshold)[0]
print(f"Potential outliers: {outliers}")


Potential outliers: [ 6 13 25 67 79]
